# Project 2 - Source coding, data compression and channel coding

The goal of this second project is to apply some of the principles seen in the lectures about source coding, data compression and channel coding. We ask you to write a brief report (pdf format) collecting your answers to the different questions. All codes must be written in Python inside this Jupyter. Note that you can not change the content of locked cells or import any extra Python library than the ones already imported (numpy).

## Implementation

In this project, you will need to use implement source coding algorithms to answer several questions. Therefore, in this first part, you are asked to write several functions that implement two of the  algorithms seen in the theoretical lectures and one new algorithm described in the project statement. Remember that you need to fill in this Jupyter Notebook to answer these questions. Pay particular attention to the required input and output format of each function.

In [3]:
# [Locked Cell] You can not import any extra Python library in this Notebook.
import numpy as np

### Question 1
Implement a function that returns a binary Huffman code for a given probability distribution. Give the main steps of your implementation. Verify your code on Exercise 7 of the second exercise session (TP2), and report the output of your code for this example. Explain how to extend your function to generate a Huffman code of any (output) alphabet size. 


In [ ]:
def Huffman_code(probability_dict):
    """
    Create the Huffman code for given probabilities  
    
    Arguments:
    ----------
    probability_dict:
      - keys: symbol as character or string
      - values: probability of the symbol as a float or double
      Example: {"A": 0.25, "B":0.5,"C":0.125,"D":0.125}
    
    Return:
    -------
    - codewords: dictionary with the name and the corresponding codeword 
      - keys: symbol as character or string
      - values: associated codeword as a character or a string    
      Example: {"A": "10", "B":"0","C":"111","D":"110"}
    
    """  

### Question 2

Given a sequence of symbols, implement a function that returns a dictionary and the encoded sequence using the on-line Lempel-Ziv algorithm (see State of the art in data compression, slide 50/53). Reproduce and report the example given in the course. 

In [ ]:

def LZ_online(sequence):
    """
    The on-line Lempel-Ziv algorithm given a sequence of symbols  
    Arguments:
    ----------
    - sequence : Sequence of symbols in the string format
    
    Return:
    -------
    - dictionary : the computed dictionnary in the form:
        - keys: symbol as character or string
        - values: associated codeword as a tuple composed of the entry index (integer) and a binarized adress with one appended symbol (character or string)
        Example: {'': (0, ''), '0': (1, '0'), '1': (2, '01'), '00': (3, '010'), '10': (4, '100')}
    - encoded_sequence : the encoded sequence in the string format
    """

### Question 4

Implement a function that returns the encoded sequence using the LZ77 algorithm as described by the algorithm below given an input string and a sliding window size l. Reproduce the example given in Figure 2 with window_size=7.

In [ ]:
def LZ77(sequence, window_size=7):
    """
    The Lempel-Ziv 77 algorithm given a sequence of symbols and the sliding window size
    
    Arguments:
    ----------
    - sequence : Sequence of symbols in the string format
    - window_size : sliding window size as an integer
    
    Return:
    -------
    - encoded_sequence : the encoded sequence in the string format
    """

In [ ]:
# [Locked Cell] Evaluation of your functions by the examiner. 
# You don't have access to the evaluation, this will be done by the examiner.
# Therefore, this cell will return nothing for the students.
import os
if os.path.isfile("private_evaluation.py"):
    from private_evaluation import unit_tests
    unit_tests(Huffman_code, LZ_online, LZ77)

## Source coding and reversible (lossless) data compression


In [ ]:
# Write here your codes for questions 5 to 15 (you may delete this comment)

## Channel coding

In [10]:
# Write here your codes for questions 16 to 21 (you may delete this comment)
# From here, you may import either opencv (cv2) or the Python Imaging Library (PIL), but no other extra libraries.

from PIL import Image,ImageOps
import sys
import numpy as np

def read_greyscale_image(path):

    img = Image.open(path) 
    if img is None:
        sys.exit("Could not read the image.")

    grey_scale = ImageOps.grayscale(img)
    grey_scale.show() 

    return grey_scale

def save_image(img,path):
    if img is None or path is None:
        sys.exit("Could not save Image.")

    img.save(path, "PNG")

    return

def encode_image(img):
    return np.matrix.flatten(np.unpackbits(np.asarray(img),axis = 1))

def decode_image(image_as_bits,shape):
    return np.reshape(np.packbits(image_as_bits,axis = 0),shape)

def hamming_sequence_encoding(sequence):
    #pad the sequence to be a multiple of 4
    padding_len = len(sequence) % 4
    np.append(sequence,np.zeros(padding_len, dtype=np.int8))

    #apply hamming code to every 4 char
    hamming_sequence = np.zeros(len(sequence)//4*7,dtype=np.int8)
    for i in range(0,len(sequence),4):
        code = hamming_code(sequence[i:i+4])

        hamming_sequence[((i//4)*7):((i//4)*7)+7] = code

    return hamming_sequence

def hamming_sequence_decoding(sequence,original_nb_bits):
    decoded_sequence = np.zeros(len(sequence)//7 * 4,dtype=np.int8)
    nb_corrections = 0 

    for i in range(0,len(sequence),7):
        decoded_bits = decode_hamming(sequence[i:i+7])
        if np.array_equal(decoded_bits, sequence[i:i+4]):
            None
        else :
            nb_corrections += 1

        decoded_sequence[(i//7) *4 : ((i//7) * 4) + 4] = decoded_bits
        
    #remove padding done at source
    return decoded_sequence[0:original_nb_bits]

def sequence_through_channel(original_sequence):
    noisy_sequence = original_sequence

    for i, bit in enumerate(noisy_sequence):
        noisy_sequence[i] = noisy_channel(bit)

    return noisy_sequence

def noisy_channel(bit):
    if np.random.rand() > 0.01 :
        return bit
    else:
        return (bit + 1) % 2

def hamming_code(bits):
    if len(bits) != 4:
        print('error: wrong number of bits given')
        return None

    parity = np.zeros(3,dtype=np.int8)
    for i in range(len(parity)):
        parity[i] = (bits[i%4] + bits[(i+1)%4] + bits[(i+2)%4]) % 2

    code = np.append(bits,parity)
    return code

def decode_hamming(code):
    if len(code) != 7:
        print('error: wrong number of bits given')
        return None

    bits = np.copy(code[0:4])
    received_parity = code[4:7]
    computed_parity = (hamming_code(code[0:4]))[4:7]

    syndrome = np.zeros(3,dtype=np.int8)
    nb_errors = 0 

    for i in range(len(received_parity)):

        if(computed_parity[i] != received_parity[i]):
            syndrome[i] = 1
            nb_errors += 1

    if nb_errors == 3:
        bits[2] = (code[2] + 1) % 2

    if nb_errors == 2:
        if syndrome[0] and syndrome[1]:
            bits[1] = (code[1] + 1) % 2

        if syndrome[1] and syndrome[2]:
            bits[3] = (code[3] + 1) % 2

        if syndrome[2] and syndrome[0]:
            bits[0] = (code[0] + 1) % 2

    return bits


def number_of_differences(seq1,seq2):
    nb_diff = 0
    for i in range(len(seq1)):
        if seq1[i] != seq2[i]:
            nb_diff += 1
    
    return nb_diff

def naive_hamming_seq_decoding(sequence,original_nb_bits):

    decoded_sequence = np.zeros(len(sequence)//7 * 4,dtype=np.int8)

    for i in range(0,len(sequence),7):
        decoded_sequence[(i//7) *4 : ((i//7) * 4) + 4] = sequence[i:i+4]
        
    #remove padding done at source
    return decoded_sequence[0:original_nb_bits]

######################################## Code to answer the questions ##############

#### 16 :Load image and show ####################

original_image = read_greyscale_image("image.png")

#### 17 :Encode image using 1byte/pixel #########

image_as_sequence = encode_image(original_image)

#### 18 :Simulate channel and decode sequence ###
im_width, im_height = original_image.size

after_channel = decode_image(sequence_through_channel(image_as_sequence),(im_height,im_width))
Image.fromarray(after_channel).show()

save_image(Image.fromarray(after_channel),'noisy.png')

#### 19 :Encode image using Hamming code #########
hamming_sequence = hamming_sequence_encoding(image_as_sequence)

#### 20 :Simulate channel on Hamming sequence ####
hamming_after_channel = sequence_through_channel(hamming_sequence)

decoded_hamming = hamming_sequence_decoding(hamming_after_channel,len(image_as_sequence))
naive_decoded_hamming = naive_hamming_seq_decoding(hamming_after_channel,len(image_as_sequence))

print("errors without hamming decoding compared to original image = " + str(number_of_differences(image_as_sequence, naive_decoded_hamming)))
print("errors with  hamming decoding compared to original image = " + str(number_of_differences(image_as_sequence, decoded_hamming)))

Image.fromarray(decode_image(decoded_hamming,(im_height,im_width))).show()
save_image(Image.fromarray(decode_image(decoded_hamming,(im_height,im_width))),'postHamming_decoded.png')
save_image(Image.fromarray(decode_image(naive_decoded_hamming,(im_height,im_width))),'naive_Hamming_decoding.png')


<PIL.PngImagePlugin.PngImageFile image mode=P size=1984x1116 at 0x7FBD969F5070>


Segmentation fault (core dumped)
